# This notebook serves as walkthrough for planning an experiment for creation through the OT2.
### The following modules are used and should be in the same directory as this notebook: 
* **CreateSamples** is responsible for calculating sample information, which includes component weight fractions and stock volumes
* **OT2Commands** is responsible for setting up information to be interpretted and executed by opentrons **BASED ON THE LATEST API 2.3 and above**
* **OT2Graphing** contains graphing tools to help visualize and explore parameter spaces.

In [2]:
from Plan import CreateSamples
from Prepare import OT2Commands as ALH
from Prepare import OT2Graphing as ographing
from opentrons import simulate, execute, protocol_api
import importlib
import pandas as pd

In [3]:
importlib.reload(ALH)

<module 'Prepare.OT2Commands' from 'C:\\Users\\Edwin\\Desktop\\OT2-DOE\\PlanPrepareProcess\\Prepare\\OT2Commands.py'>

### Step 1: Set up the experiment dictionary.
The first step to planning an experiment is to load the experiment variables and inputs from a csv file. **Within the Testing Plans Folder you should find a templete of a testing plan (open in Excel for easier viewing).**

Currently the info is split up between two sections: Variables for creating sample spaces and variables for OT2 commands. You only need to specify for the sections you will use.The experiment dictionary consist of keys being the variable name and the value being the variables value. Keep in mind the order of listed variables assumptions should be the index of one listed variable refers to the same index of a related variable.

Besides the plan, if planning on creating a sample space you will need to:
- Update/verify the csv chemical database with/for component information.
- Provide the path of the updated chemical csv for plan. 

In [4]:
path = r"C:\Users\Edwin\Desktop\OT2-DOE\PlanPrepareProcess\Testing Plans\07_08_21_Octadecene_Ethanol.csv"
chem_path = r"Chemical Database.csv"
plan = CreateSamples.get_experiment_plan(path, chem_path)
plan

C:\Users\Edwin\Desktop\OT2-DOE\PlanPrepareProcess\Plan\CreateSamples.py:28: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  plan_dict['Chemical Database'] = chem_data.T.to_dict()


{'Sample Amount': 1.2,
 'Sample Unit': 'g',
 'Component Shorthand Names': ['ODE', 'SDS', 'ethanol', 'water'],
 'Component Concentration Units': ['wtf', 'wtf', 'wtf', 'wtf'],
 'Component Concentration Linspaces [min, max, n]': [[0, 0.001, 10],
  [0.0005, 0.0005, 1],
  [0.1, 0.8, 8]],
 'Stock Names': ['ODE-ethanol-stock',
  'SDS-ethanol-stock',
  'ethanol-stock',
  'water-stock'],
 'Stock Concentration Units': ['mgpermL', 'mgpermL', 'wtf', 'wtf'],
 'Stock Concentrations': [5, 10, 1, 1],
 'Stock Densities (g/mL)': [0.789, 0.789, 0.789, 1],
 'OT2 Destination Labwares': ['abgene_96_wellplate_2200ul'],
 'OT2 Destination Labware Slots': [2],
 'OT2 Stock Labwares': ['20mlscintillation_12_wellplate_18000ul'],
 'OT2 Stock Labware Slots': ['1'],
 'OT2 Right Pipette': 'p300_single_gen2',
 'OT2 Right Pipette Aspiration Rate (uL/sec)': 50,
 'OT2 Right Pipette Dispense Rate (uL/sec)': 300,
 'OT2 Right Tipracks': ['opentrons_96_tiprack_300ul',
  'opentrons_96_tiprack_300ul'],
 'OT2 Right Tiprack Slots

### Step 2: Select and Create Sampling Space
The sampling method will depend on the type of sample units, the actual called method (i.e. lattic, non-clustering random..etc) and whether you are providing arguments from the plan in step 1 or importing sample information from elsewhere. 

***Note:*** All information will be stored as a dataframe
 
You can utilize the built in functions in functions to create sample concentration spaces as follows:
- concentration_from_csv: Import csv with headers being component name and concentration unit.
- concentration_from_excel: Import from excel with headers being component name and concentration unit.
- concentration_from_linspace: Pull from plan the linspaces to use lattice or random sampling. Optional: Unity filter allows for last indexed component to be used a completing component. 
- concentration_from_list_samplewise: 
- concentration_from_list_componentwise:
 
**The ONE requirement if you plan to use any of the module functions is you adhere to strict naming convention. This allows the functions to identify key values to use to calculate and determine specific informaiton (i.e. mass, volume to pipette). **

The following conventions are:
- component concentration columns: Should have the term "concentration" and the concentration unit name (i.e. DPPC Concentration Molarity). 
-

In [5]:
concentration_df = CreateSamples.concentration_from_linspace(plan['Component Shorthand Names'], 
                                                                     plan['Component Concentration Linspaces [min, max, n]'],
                                                                     plan['Component Concentration Units'], unity_filter= True)        
concentration_df

,ODE concentration wtf,SDS concentration wtf,ethanol concentration wtf,water concentration wtf
0,0.000,0.0005,0.1,0.8995
1,0.000,0.0005,0.2,0.7995
2,0.000,0.0005,0.3,0.6995
3,0.000,0.0005,0.4,0.5995
4,0.000,0.0005,0.5,0.4995
...,...,...,...,...
75,0.001,0.0005,0.4,0.5985
76,0.001,0.0005,0.5,0.4985
77,0.001,0.0005,0.6,0.3985
78,0.001,0.0005,0.7,0.2985


### Step 3: Create Stock Dictionary
The stock dictionary will hold the stock information which will be used to calculate for stock volumes for the OT2.
The stock dictionary function will take 3 main sets of information (all formatted as lists, matched by index). 
- Stock Names: Stock names are to be formatted as solute1-solute2...soluten-solvent-stock i.e. NaCl-water-stock.
    - The solutes and solvents are expected to be in the chemical dictionary provided in step 1 as information such as density and molecular weight will be used in volume calculations.
- Stock Concentration Units: Any conc units supported (see function).
- Stock Concentrations: Value of stock conc.

In [6]:
stock_dict = CreateSamples.stock_dictionary(plan['Stock Names'], 
                                                    plan['Stock Concentration Units'], 
                                                    plan['Stock Concentrations'], stock_densities=plan['Stock Densities (g/mL)'])
stock_dict

{'ODE-ethanol-stock': {'solutes': ['ODE'],
  'solvents': 'ethanol',
  'unit': 'mgpermL',
  'concentration': 5,
  'Density (g/mL)': 0.789,
  'Common Solvent': 'Mixture'},
 'SDS-ethanol-stock': {'solutes': ['SDS'],
  'solvents': 'ethanol',
  'unit': 'mgpermL',
  'concentration': 10,
  'Density (g/mL)': 0.789,
  'Common Solvent': 'Mixture'},
 'ethanol-stock': {'solutes': [],
  'solvents': 'ethanol',
  'unit': 'wtf',
  'concentration': 1,
  'Density (g/mL)': 0.789,
  'Common Solvent': 'Pure'},
 'water-stock': {'solutes': [],
  'solvents': 'water',
  'unit': 'wtf',
  'concentration': 1,
  'Density (g/mL)': 1,
  'Common Solvent': 'None'}}

### Step 3: Calculate Component Amounts Mass And Volumes
The end goal in this component planning is to calculate for stock volumes, this requires knowledge on the component mass/volumes.

For now there exist one general function *determine_component_amounts* which based on the total amount (either volume or mass) and component conc units will use the appropiate function to calculate for component mass and volume amounts. 

Since this step uses information from the chemical database (density and mw), if that information is missing or not avaible then the resulting values will return as nan. To remove these and replace with zero in cases where the component mass/volume is negligible use the argument within the function to specify a fill amount of nan_fill_value = 0.

In [7]:
complete_amounts_added = CreateSamples.determine_component_amounts(plan, concentration_df, nan_fill_value=0)
complete_amounts_added

You calculated for component masses given the provided units
You calculated for component masses given the provided units
You calculated for component masses given the provided units
You calculated for component masses given the provided units


,ODE concentration wtf,SDS concentration wtf,ethanol concentration wtf,water concentration wtf,ODE amount mass g,ODE amount volume mL,SDS amount mass g,SDS amount volume mL,ethanol amount mass g,ethanol amount volume mL,water amount mass g,water amount volume mL
0,0.000,0.0005,0.1,0.8995,0.0000,0.000000,0.0006,0.000594,0.12,0.152033,1.0794,1.0794
1,0.000,0.0005,0.2,0.7995,0.0000,0.000000,0.0006,0.000594,0.24,0.304067,0.9594,0.9594
2,0.000,0.0005,0.3,0.6995,0.0000,0.000000,0.0006,0.000594,0.36,0.456100,0.8394,0.8394
3,0.000,0.0005,0.4,0.5995,0.0000,0.000000,0.0006,0.000594,0.48,0.608134,0.7194,0.7194
4,0.000,0.0005,0.5,0.4995,0.0000,0.000000,0.0006,0.000594,0.60,0.760167,0.5994,0.5994
...,...,...,...,...,...,...,...,...,...,...,...,...
75,0.001,0.0005,0.4,0.5985,0.0012,0.001521,0.0006,0.000594,0.48,0.608134,0.7182,0.7182
76,0.001,0.0005,0.5,0.4985,0.0012,0.001521,0.0006,0.000594,0.60,0.760167,0.5982,0.5982
77,0.001,0.0005,0.6,0.3985,0.0012,0.001521,0.0006,0.000594,0.72,0.912201,0.4782,0.4782
78,0.001,0.0005,0.7,0.2985,0.0012,0.001521,0.0006,0.000594,0.84,1.064234,0.3582,0.3582


### Step 4: Calculate Volumes of Stocks
From the mass/volume of individual chemical components we calculauted along with the concentration and identities of the provided stocks, we are now able to combine those two pieces of information and obtain the volume of stock needed. 

The general function to use Create.calculate_stock_volumes_from_component_concs will autoamatically use the component information either volume or stock depending on the stock which contains the component and its stock unit. The stock unit essentially determines the pathway of whether component mass or volume will be utilized.See ______ for a map of how specific component concentration untis and stock units pair together to calculate for info. 

Note: This will calculate stock volumes not taking into account any interdepdencies such as common solvents or missing volume to complete a desired sample volume. The next step will have a few cases for which specfic functions can calculate and adjust stock volumes. 

In [8]:
stock_volumes_prelim = CreateSamples.calculate_stock_volumes_from_component_concs(plan, complete_amounts_added, stock_dict)
stock_volumes_prelim

,ODE concentration wtf,SDS concentration wtf,ethanol concentration wtf,water concentration wtf,ODE amount mass g,ODE amount volume mL,SDS amount mass g,SDS amount volume mL,ethanol amount mass g,ethanol amount volume mL,water amount mass g,water amount volume mL,ODE-ethanol-stock amount volume mL,SDS-ethanol-stock amount volume mL,ethanol-stock amount volume mL,water-stock amount volume mL
0,0.000,0.0005,0.1,0.8995,0.0000,0.000000,0.0006,0.000594,0.12,0.152033,1.0794,1.0794,0.00,0.06,0.152091,1.0794
1,0.000,0.0005,0.2,0.7995,0.0000,0.000000,0.0006,0.000594,0.24,0.304067,0.9594,0.9594,0.00,0.06,0.304183,0.9594
2,0.000,0.0005,0.3,0.6995,0.0000,0.000000,0.0006,0.000594,0.36,0.456100,0.8394,0.8394,0.00,0.06,0.456274,0.8394
3,0.000,0.0005,0.4,0.5995,0.0000,0.000000,0.0006,0.000594,0.48,0.608134,0.7194,0.7194,0.00,0.06,0.608365,0.7194
4,0.000,0.0005,0.5,0.4995,0.0000,0.000000,0.0006,0.000594,0.60,0.760167,0.5994,0.5994,0.00,0.06,0.760456,0.5994
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,0.001,0.0005,0.4,0.5985,0.0012,0.001521,0.0006,0.000594,0.48,0.608134,0.7182,0.7182,0.24,0.06,0.608365,0.7182
76,0.001,0.0005,0.5,0.4985,0.0012,0.001521,0.0006,0.000594,0.60,0.760167,0.5982,0.5982,0.24,0.06,0.760456,0.5982
77,0.001,0.0005,0.6,0.3985,0.0012,0.001521,0.0006,0.000594,0.72,0.912201,0.4782,0.4782,0.24,0.06,0.912548,0.4782
78,0.001,0.0005,0.7,0.2985,0.0012,0.001521,0.0006,0.000594,0.84,1.064234,0.3582,0.3582,0.24,0.06,1.064639,0.3582


### Step 4a: Adjust/Add Stock Volumes

Case 1: Common solvents, in this case solvents are shared between more than one stock the stock needs to be taken into account if provided as component in the first place. For example if ethanol is present in 2 stocks and ethanol must be a specfic wtf then we must take into account the ethanol added when adding those two stocks. The case may arise where you do not specify the concentration of the common solvent and in that case we do not care about the final concentration of common solvent so no need to track for it. The function you would use is **Create.calculate_common_solvent_residual_volumes()**

Case 2: Filling case, in this case after calculating each stock volume for a sample it may need to be diluted/have the total sample volume completed by a component. The function to use is **Create.complete_missing_volume_with_commmon_solvent()** which you can specify the name of a new stock along with the total completing volume and/or if not provided will automatically look for a single solvent stock and use that to fill.

Case X: Other?

***NOTE: ALWAYS CONVERT THE FINAL mL TO uL AS OT2 NATIVELY READS ALL NUMBERS AS uL USE THE CreateSamples.convert_mL_to_uL() FUNCTION ***

In [9]:
# stock_volumes_completed = CreateSamples.complete_missing_volume_with_commmon_solvent(plan['Sample Amount'],stock_volumes_prelim, stock_dict, 'Octadecene')
stock_volumes_completed = CreateSamples.calculate_common_solvent_residual_volumes(stock_volumes_prelim, stock_dict)
complete_df = CreateSamples.convert_mL_to_uL(stock_volumes_completed)

### Step 5: Apply Filters or Other Selection Criteria to Stock Volumes

**Filters**: The types of filters can be added easily through dataframe logic. For example, ones in the CreateSamples module:
   * *filter_general_min_pipette_restriction* = Given the lower of the pipettes you will use to synthesize the samples, will remove anything not in range or zero. 
   * *filter_total_volume_restriction* = Restrict total volume, mostly based on labware constraints. Column selection based on "total". 
   * *filter_general_max_restriction* = provide max value and column name 
   
**Other Selection Critera**
   * *.multiplydf(n, df)* = Will multiply a data frame by n number of times. Useful when making one sample repeatadly.

In [10]:
complete_df_filtered = CreateSamples.filter_general_min_pipette_restriction(complete_df, 20)
complete_df_filtered = CreateSamples.filter_total_volume_restriction(complete_df_filtered,1600)
complete_df_filtered = CreateSamples.filter_general_max_restriction(complete_df_filtered,1000, 'water-stock amount volume uL')
complete_df_filtered.reset_index(inplace=True,drop=True)
complete_df_filtered = complete_df_filtered[0:4]

### Step 5a (Optional): Visual
If you would like, you can visual the space you are sampling by calling up to 3 columns of interest from the componsitional space. The module OT2Graphing will contain helpful function to visualize the sampling space.
* Note: Given limitation with adding packages to OT2, anything utilziing matplotlib will not work on the OT2's native jupyter notebook. This is currently the only module which references it. 

In [11]:
ographing.xy_scatter_df_compare(t, f, 'ethanol concentration wtf', 'ODE concentration wtf')

NameError: name 't' is not defined

### Step 6 (Optional): Calculate Stock Prep Information

We can calculate stock information to make it easier to prepare these stock. This process if semi-general, currently supporting a few stock concentration units = mgperml, volf, wtf, molf and molarity. All of these require the basis of volume as that is what is driving how much sample we make for the OT2. 

The logic behind this relies on the notation of stocks = solute1-solute2-....solvent. Using notations allows for systematic handling of solutes and solvent determining what is needed based on information from a excel based Chemical Database.

*Reccomended*: Use at the minimum the default buffering (extra stock being made) of 40 percent to ensure you do not aspirate any air by mistake.

BROKEN FIXING

In [12]:
# stock_volumes = CreateSamples.isolate_common_column(complete_df_filtered, 'stock')
# chem_database_path = r"Chemical Database.xlsx"
# stock_prep_df = CreateSamples.calculate_stock_prep_df(plan, stock_volumes, chem_database_path, buffer_pct=50)
# stock_prep_df
stock_volumes = CreateSamples.isolate_common_column(complete_df_filtered,'stock').sum(axis=0)/1000
stock_volumes

ODE-ethanol-stock amount volume uL    0.000000
SDS-ethanol-stock amount volume uL    0.240000
ethanol-stock amount volume uL        1.891654
water-stock amount volume uL          3.117600
dtype: float64

### Step 8: Simulate/Execute
The main information needed to create commands for the OT2 is the position of the stocks and the volumes in respect to each well you would like to move the stock to. All volumes in sent to the OT2 will be interpretted as microliters (uL). 

* Pipetting Strategy 1: Add each stock to all wells then move onto next stock. The upside to this is the simplicity of the written protocol. It has a downside of using more tips. It can allow for evaporation or other time sensitve process to occur in the time between stock additions. This is referred to component wise pipetting.

* Pipetting Strategy 2: Adding all stocks to a ONE well then moving on, this is referred to as sample wise pipetting. 

### Step 8a Optional: Load custom labware dictionary (Remote Testing)
* Provide the path to the directory holding all custom labware. This directory should have custom labware .json files you have previously made and tested, read more here: https://support.opentrons.com/en/articles/3136504-creating-custom-labware-definitions
* The reason we provide this is when working on a device that is not connected to the OT2's Jupyter notebook there is no way to natively use custom labware. So we create a dictionary of custom labware so we can later load into our protocol to primarily simualte and test protocols for execution later once connnected to the OT2's notebook.
    * When using custom labware on the OT2's notebook it pulls from a folder labeled "labware", which is something built into the Opentrons hardware. It has not been tested if the custom labware dictionary will superceed this directory of custom labware if used on the OT2.
    * The custom labware folder should contain everything the OT2 labware folder contains: ADD GDRIVE link to customize
    
* **NOTE: Any function using the custom_labware_dict, the argument is optional so can remove and should receive same output IF working on OT2 computer.**

Just noted: transfer_info will fail if transfer and distribute used together, well not fail jsut need to concat

In [13]:
labware_dir_path = r"Custom Labware"
custom_labware_dict = ALH.custom_labware_dict(labware_dir_path)

### Step 9: Set up Loaded Dictionary and Directions for Volume Handling
It is important to know how many stock container will be needed for each stock and be able to provide the correct instructions for OT2 commands to follow.

The ranges of stock refers to the ranges of wells one stock will cover. The range is provided in terms of the index of the stock volume dataframe i.e.([lower, upper])). The format is as follows = [[stock1_range1, stock1_range2....], [stock2_range1, stock2_range2....] []...]. 

The range is based on the provided maximum volume for each container, since variabiltiy exist between filling it is **HIGHLY RECCOMENDED** to underestimate the total volume of stock a container can hold so for a 20mL scitillation vials 16000-17000 should be use as the max. Another reason for this is the OT2 will do its best to pick up as much volume as possible but understand liquid in vials can coat the sides not leaving enough depth for the OT2 to grab, so it is **HIGHLY RECCOMENDED** to have at least a **15-20%** volume buffer for each stock

The basis of range is on the maximum volume of the stock container, however currently limitation are: 
 * All stock containers must be the same

In [48]:
importlib.reload(ALH)

<module 'Prepare.OT2Commands' from 'C:\\Users\\Edwin\\Desktop\\OT2-DOE\\PlanPrepareProcess\\Prepare\\OT2Commands.py'>

In [44]:
protocol = simulate.get_protocol_api('2.8', extra_labware=custom_labware_dict)
loaded_dict = ALH.loading_labware(protocol, plan, well_order='row')
ALH.add_labware_to_dict(loaded_dict, 'Stock Wells Large', ['agilent_6_reservoir_47000ul'], [6])
ALH.add_labware_to_dict(loaded_dict, 'Cleaning Wells', ['agilent_6_reservoir_47000ul'], [9])
stock_position_info = ALH.stock_well_ranges(complete_df_filtered, loaded_dict['Stock Wells'])
water = ALH.stock_well_ranges(complete_df_filtered['water-stock amount volume uL'], loaded_dict['Stock Wells Large'])
ethanol = ALH.stock_well_ranges(complete_df_filtered['ethanol-stock amount volume uL'], loaded_dict['Stock Wells Large'])

stock_position_info.update(water)
stock_position_info.update(ethanol)
stock_position_info

C:\Users\Edwin\.opentrons\robot_settings.json not found. Loading defaults
C:\Users\Edwin\.opentrons\deck_calibration.json not found. Loading defaults


{'ODE-ethanol-stock amount volume uL': {'Ranges': [[0, 4]],
  'Total Volume': 0.0,
  'Stock Wells': [A1 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1]},
 'SDS-ethanol-stock amount volume uL': {'Ranges': [[0, 4]],
  'Total Volume': 239.99999999999997,
  'Stock Wells': [A2 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1]},
 'ethanol-stock amount volume uL': {'Ranges': [[0, 4]],
  'Total Volume': 1891.6538041636863,
  'Stock Wells': [A1 of Agilent 6 Reservoir 47000 ÂµL on 6]},
 'water-stock amount volume uL': {'Ranges': [[0, 4]],
  'Total Volume': 3117.6,
  'Stock Wells': [A1 of Agilent 6 Reservoir 47000 ÂµL on 6]}}

In [45]:
ALH.cleaning_tip_protocol(loaded_dict, 2) # can do something like this to add other kwargs prior and after trasnfer

Enter Well Index 0
Adding A1 of Agilent 6 Reservoir 47000 ÂµL on 9 for washing step
Enter Cleaning Delay 10
Enter Times Wanting to mix 3
Enter Well Index 1
Adding A2 of Agilent 6 Reservoir 47000 ÂµL on 9 for washing step
Enter Cleaning Delay 30
Enter Times Wanting to mix 3


{'Cleaning 0': {'well': A1 of Agilent 6 Reservoir 47000 ÂµL on 9,
  'delay': '10',
  'mix_n': 3},
 'Cleaning 1': {'well': A2 of Agilent 6 Reservoir 47000 ÂµL on 9,
  'delay': '30',
  'mix_n': 3}}

In [46]:
directions = ALH.create_sample_making_directions(complete_df_filtered, stock_position_info, loaded_dict, start_position=0)
directions

{0: {'ODE-ethanol-stock amount volume uL': {'Stock Position': A1 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1,
   'Destination Well Position': A1 of Abgene 96 Well Plate 2200 ÂµL on 2,
   'Stock Volume': 0.0},
  'SDS-ethanol-stock amount volume uL': {'Stock Position': A2 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1,
   'Destination Well Position': A1 of Abgene 96 Well Plate 2200 ÂµL on 2,
   'Stock Volume': 59.99999999999999},
  'ethanol-stock amount volume uL': {'Stock Position': A1 of Agilent 6 Reservoir 47000 ÂµL on 6,
   'Destination Well Position': A1 of Abgene 96 Well Plate 2200 ÂµL on 2,
   'Stock Volume': 244.776568911644},
  'water-stock amount volume uL': {'Stock Position': A1 of Agilent 6 Reservoir 47000 ÂµL on 6,
   'Destination Well Position': A1 of Abgene 96 Well Plate 2200 ÂµL on 2,
   'Stock Volume': 959.3999999999999}},
 1: {'ODE-ethanol-stock amount volume uL': {'Stock Position': A1 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1,
   'Destination Well Posi

In [47]:
ALH.pipette_volumes_sample_wise(protocol, directions, loaded_dict, reuse_tips= True, clean_tips=True, touch_tip = True)

TypeError: unsupported type for timedelta seconds component: str

### Step 10: Format for Exporting and Upload to Google Drive
* Using the information from our previously created dataframes we create a final dataframe to convert to a csv and upload to google drive for storage. 
    * Currently the two main pieces of information uploaded are composition of sample and sample location information. This for each sample is tied to a unique ID which contains a well_timestamp_keyword. 

* Currently the function *CreateSamples.add_final_location* uses direciton information plus the sampling dataframe either created or imported to add useful information and export as a csv.

In [ ]:
export_path = r"C:\Users\Edwin\Desktop\OT2-DOE\PlanPrepareProcess\Temp or Working\07_08_21_Octadecene_Info_pt2"
export_df = CreateSamples.add_final_location(directions,complete_df_filtered)
# export_df.to_csv(export_path, index = False)
export_df.reset_index(inplace=True, drop=True)
export_df

In [ ]:
export_df.to_csv(export_path, index=False)

In [ ]:
def function(arg1, **kwargs):
    print(arg1)
    for key, value in kwargs.items():
        print("{0} = {1}".format(key, value))

In [ ]:
function(1, hi = 'touch_tip')